# Running simulations in a jupyter notebook 
## Prerequisites
1. Install the environment contained in the main folder using 
```
conda env create -f modelling.yml 
```
2. Open the jupyter notebook in your conda environment, quite simple in VS Code but also straight forward in conda from terminal.

*Important:* Remember to have already installed GROMACS in your laptop before proceeding. Otherwise come back to [README](../../README.md)

## Loading all the modules

We load the libraries `os` that allows us to execute commands in terminal from the Notebook

In [9]:
import os

In [10]:
import nglview as nv
!jupyter-nbextension enable nglview --py --sys-prefix # Needed to ensure you can visualize nglview objects in the notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import py3Dmol
import MDAnalysis as mda

Enabling notebook extension nglview-js-widgets/extension...
      - Validating: OK


### Calling GROMACS from terminal

### Making the topology

In [11]:
!printf '15' | gmx pdb2gmx -f 1aki -o 1_aki_proc -water spce

                 :-) GROMACS - gmx pdb2gmx, 2021.3-bioconda (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff            Tee

### Creating the simulation box

In [ ]:
!gmx editconf -f 1aki_proc -c -d 0.5 -bt triclinic -o box

### Solvating the box

In [ ]:
!gmx solvate -cp box -cs spc216 -p topol -o solv

## Adding ions

In [ ]:
!gmx grompp -f ions -c solv -p topol -o ions

In [ ]:
!printf '13' | gmx genion -s ions -p topol -neutral -pname NA -nname CL -o solv_ions

## Energy minimization

In [ ]:
!gmx grompp -f em -c solv_ions -p topol -o em
!gmx mdrun -v -deffnm em

## Plotting properties

In [ ]:
!printf "10 0" | gmx energy -f em -o potential
potential = np.genfromtxt([i for i in open('potential.xvg').read().splitlines()
    if not i.startswith(('#','@'))])
plt.plot(*potential.T)
plt.xlabel('step')
plt.ylabel('potential')

## NVT Equilibration

In [ ]:
!gmx grompp -f nvt -c em -p topol -o nvt
!gmx mdrun -v -deffnm nvt

### Plotting pressure evolution during the NVT

In [ ]:
!printf '17' | gmx energy -f nvt -o pressure
pressure = np.genfromtxt([i for i in open('pressure.xvg').read().splitlines()
    if not i.startswith(('#','@'))])
plt.plot(*pressure.T)
plt.xlabel('step')
plt.ylabel('pressure')

## NPT Equilibration

In [ ]:
!gmx grompp -f npt -c nvt -p topol -o npt
!gmx mdrun -v -deffnm npt

## MD Production

In [ ]:
!gmx grompp -f md -c npt -p topol -o md
!gmx mdrun -v -deffnm md

## Visualization

### With py3Dmol

In [16]:
aki = py3Dmol.view('md.gro')
aki.setStyle('cartoon')
aki.insert('md.xtc')
aki.show

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

<bound method using_ipython.<locals>.inner of <py3Dmol.view object at 0x7fa782aa1d90>>

### With NGLview

In [14]:
import nglview as nv
import mdtraj as md
traj = md.load('md.xtc',top='md.gro')
t = nv.MDTrajTrajectory(traj)
w = nv.NGLWidget(t)
w

NGLWidget(count=101)

### With MDAnalysis
Allows in a very simple way visualize trajectories

In [15]:
u = mda.Universe('md.gro','md.xtc')
w = nv.show_mdanalysis(u)
w

/Users/trivino/opt/miniconda3/envs/modelling/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:1080: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  "".format(attrname, default))
/Users/trivino/opt/miniconda3/envs/modelling/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:1080: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  "".format(attrname, default))
/Users/trivino/opt/miniconda3/envs/modelling/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:1080: UserWarning: Found no information for attr: 'chainIDs' Using default value of ''
  "".format(attrname, default))
/Users/trivino/opt/miniconda3/envs/modelling/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:1080: UserWarning: Found no information for attr: 'occupancies' Using default value of '1.0'
  "".format(attrname, default))
/Users/trivino/opt/miniconda3/envs/modelling/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:1080

NGLWidget(count=101)